In [ ]:
import numpy as np

In [ ]:
from scipy.stats import chi2,ncx2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
np.random.seed(0)

In [ ]:
def model(xi,biasIndex):
    #if biasIndex = 0, we assume no bias
    #if biasIndex not 0, its value is where bias is injected
    if biasIndex == 0:
        m = xi.shape[1]
        H = np.concatenate((xi,np.ones((1,m)))).T
        return H
    else:
        m = xi.shape[1]
        H = np.concatenate((xi,np.ones((1,m)),np.zeros((1,m)))).T
        H[biasIndex-1,2] = 1.
        return H

In [ ]:
def measurement(H,X,R,bias):
    #generates measurements and adds a bias
    (m,n) = H.shape
    Y = H@X+np.linalg.cholesky(R)@np.random.randn(m,1) + bias
    return Y

In [ ]:
def projector(H,R):
    #returns (I-Htilde*Stilde)
    L = np.linalg.cholesky(R)
    (m,n) = H.shape
    Htilde = np.linalg.solve(L,np.eye(m))@H
    Stilde = np.linalg.solve((Htilde.T@Htilde),np.eye(n))@(Htilde.T)
    P = np.eye(m)-Htilde@Stilde
    return P

In [ ]:
def residus(Y,P,R):
    #Y : measurements
    #P : projector
    #R : noise covariance matrix
    L = np.linalg.cholesky(R)
    (m,m) = P.shape
    Ytilde = np.linalg.solve(L,np.eye(m))@Y
    res = (Ytilde.T@P@Ytilde)
    return res

In [ ]:
def modelEstimationAndLikelihood(xi,Y,biasIndex,R):
    H = model(xi,biasIndex)
    L = np.linalg.cholesky(R)
    (m,n) = H.shape
    Htilde = np.linalg.solve(L,np.eye(m))@H
    Stilde = np.linalg.solve((Htilde.T@Htilde),np.eye(n))@(Htilde.T)
    Ytilde = np.linalg.solve(L,np.eye(m))@Y
    Xestim = Stilde@Ytilde
    dof = m-n
    P = projector(H,R)
    res = residus(Y,P,R)
    likelihood = chi2.pdf(res,dof)
    return(likelihood,res,Xestim)

In [ ]:
def likelihoodH0H1(xi,Y,R):
    H = model(xi,0)
    L = np.linalg.cholesky(R)
    (m,n) = H.shape
    dof = m-n
    P = projector(H,R)
    res = residus(Y,P,R)
    likelihoods = np.zeros((m,2))
    for i in range(m):
        mu = np.zeros((m,1))
        mu[i:i+1,:] = 0.19
        likelihood0 = chi2.pdf(res,dof)
        likelihood1 = ncx2.pdf(res,dof,mu.T@(np.linalg.solve(L,np.eye(m)).T)@P@np.linalg.solve(L,np.eye(m))@mu)
        norm = likelihood0 + likelihood1
        likelihoods[i:i+1,0:1] = likelihood0/norm
        likelihoods[i:i+1,1:2] = likelihood1/norm
    return(likelihoods)

In [ ]:
N = 22 #number of points
#xi = np.arange(N).reshape((1,N)) #x values
#xi = np.array([0.,0.5,1.,1.5,2.,2.5,3.,3.5,4.,4.5,5.,20.]).reshape((1,N))
xi = np.concatenate((np.arange(N-1).reshape(N-1,1),np.array([[65]]))).T
#Ground truth
Xtrue = np.array([[1.],[2.]])
Htrue = model(xi,0)
R = ((2e-2)**2)*np.eye(N) #noise covariance matrix
bias = np.zeros((N,1))
bias[21,0] = 0.19 #bias introduction, python indexes at 0 !!
Y = measurement(Htrue,Xtrue,R,bias) #measurements

In [ ]:
(_,_,xest) = modelEstimationAndLikelihood(xi,Y,0,R)
x1 = xi[0,0]
x2 = xi[0,-1]
y1 = xest[0]*x1+xest[1]
y2 = xest[0]*x2+xest[1]
plt.plot([x1,x2],[y1,y2],label="Model 0")
plt.scatter(xi,Y,label="Y")
plt.legend()
plt.show()

In [ ]:
Ptrue = projector(Htrue,R)
sns.heatmap(Ptrue, linewidth=0.5)
plt.show()
diagtrue = np.diag(Ptrue)
imin = np.argmin(diagtrue)
imax = np.argmax(diagtrue)
pmin = np.min(diagtrue)
pmax = np.max(diagtrue)
print("min entry : {} index {}".format(pmin,imin))
print("max entry : {} index {}".format(pmax,imax))

In [ ]:
dof = N-2
xmin = 0.
xmax = 7*N
x=np.arange(xmin,xmax,0.01)
plt.plot(x,chi2.pdf(x,dof),label="centered")
p = 0.5
lambdancx2 = p*0.19**2/((2e-2)**2)
plt.plot(x,ncx2.pdf(x,dof,lambdancx2),label="lambda = {}, p = {}".format(lambdancx2,p))
plt.title("Chi square densities, {} dof".format(dof))
plt.legend()

Probabilité que la réalisation des résidus soit issue de la densité centrée ou non-centrée, pour chaque mesure considérée

In [ ]:
likelihoodH0H1(xi,Y,R)

We now build all possible models, and evaluate their likelihood

In [ ]:
likelihood = []
res = []
estim = []
for i in range(N+1):
    (likelihood_i,res_i,estim_i) = modelEstimationAndLikelihood(xi,Y,i,R)
    likelihood.append(likelihood_i)
    res.append(res_i)
    estim.append(estim_i)

In [ ]:
for i in range(N+1):
    x1 = xi[0,0]
    x2 = xi[0,-1]
    xest = estim[i]
    y1 = xest[0]*x1+xest[1]
    y2 = xest[0]*x2+xest[1]
    plt.plot([x1,x2],[y1,y2],label="Model {}".format(i))
plt.scatter(xi,Y,label="Y")
plt.legend()
plt.show()

In [ ]:
likelihood = np.array(likelihood).reshape(N+1)
plt.scatter(np.arange(N+1),likelihood)
plt.xlabel("Model")
plt.ylabel("Likelihood")

In [ ]:
res = np.array(res).reshape(N+1)
plt.scatter(np.arange(N+1),res)
plt.xlabel("Model")
plt.ylabel("Residuals")
plt.show()